<a href="https://colab.research.google.com/github/chaitanyagaur7/stock-market-prediction-algorithm/blob/main/Stock_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install quandl

In [3]:
import quandl
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split


In [6]:
df = quandl.get("WIKI/FB")
print(df.head())

             Open   High    Low    Close       Volume  Ex-Dividend  \
Date                                                                 
2012-05-18  42.05  45.00  38.00  38.2318  573576400.0          0.0   
2012-05-21  36.53  36.66  33.00  34.0300  168192700.0          0.0   
2012-05-22  32.61  33.59  30.94  31.0000  101786600.0          0.0   
2012-05-23  31.37  32.50  31.36  32.0000   73600000.0          0.0   
2012-05-24  32.95  33.21  31.77  33.0300   50237200.0          0.0   

            Split Ratio  Adj. Open  Adj. High  Adj. Low  Adj. Close  \
Date                                                                  
2012-05-18          1.0      42.05      45.00     38.00     38.2318   
2012-05-21          1.0      36.53      36.66     33.00     34.0300   
2012-05-22          1.0      32.61      33.59     30.94     31.0000   
2012-05-23          1.0      31.37      32.50     31.36     32.0000   
2012-05-24          1.0      32.95      33.21     31.77     33.0300   

           

In [8]:
df = df[['Adj. Close']]
df.head()

,Adj. Close
Date,
2012-05-18,38.2318
2012-05-21,34.0300
2012-05-22,31.0000
2012-05-23,32.0000
2012-05-24,33.0300


In [31]:
#A variable for predicting 'n' days in future
forecast_out = 30
df['Prediction'] = df[['Adj. Close']].shift(-forecast_out)
print(df.tail())

            Adj. Close  Prediction
Date                              
2018-03-21      169.39         NaN
2018-03-22      164.89         NaN
2018-03-23      159.39         NaN
2018-03-26      160.06         NaN
2018-03-27      152.19         NaN


In [32]:
# CREATE AN INDEPENDENT DATA SET (X)
#Convert Data Frame to numpy array
x = np.array(df.drop(['Prediction'],1))
x = x[:-forecast_out]
print(x)

[[ 38.2318]
 [ 34.03  ]
 [ 31.    ]
 ...
 [171.5499]
 [175.98  ]
 [176.41  ]]


<ipython-input-32-630fcae51f73>:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  x = np.array(df.drop(['Prediction'],1))


In [33]:
y = np.array(df['Prediction'])
y = y[:-forecast_out]
print(y)

[ 30.771  31.2    31.47  ... 159.39  160.06  152.19 ]


In [34]:
# SPLIT THE DATA TO 80% TRAINING AND 20% TESTING

x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.2)

In [35]:
#CREATE AND TRAIN THE SUPPORT VECTOR MACHINE(REGRESSOR)

svr_rbf = SVR(kernel = 'rbf', C = 1e3, gamma=0.1)
svr_rbf.fit(x_train,y_train)


SVR(C=1000.0, gamma=0.1)

In [36]:
#Testing the Model : Score returns the coefficient of determination R^2 of the prediction
#The best possible score is 1.0

svm_confindence = svr_rbf.score(x_test,y_test)
print("svm_confidence: ",svm_confindence)



svm_confidence:  0.9801072697089803


In [37]:
#Create and Train an Linear Regression Model
lr = LinearRegression()
#Train the model
lr.fit(x_train,y_train)

LinearRegression()

In [38]:
#Testing the Model : Score returns the coefficient of determination R^2 of the prediction
#The best possible score is 1.0

lr_confindence = lr.score(x_test,y_test)
print("lr_confidence: ",lr_confindence)


lr_confidence:  0.9771149941925675


In [39]:
#Set x_forecast equal to last "forecast_out" rows of the original data set from Adj_close column
x_forecast = np.array(df.drop(['Prediction'],1))[-forecast_out:]
print(x_forecast)


[[173.15]
 [179.52]
 [179.96]
 [177.36]
 [176.01]
 [177.91]
 [178.99]
 [183.29]
 [184.93]
 [181.46]
 [178.32]
 [175.94]
 [176.62]
 [180.4 ]
 [179.78]
 [183.71]
 [182.34]
 [185.23]
 [184.76]
 [181.88]
 [184.19]
 [183.86]
 [185.09]
 [172.56]
 [168.15]
 [169.39]
 [164.89]
 [159.39]
 [160.06]
 [152.19]]


<ipython-input-39-4e45ad5a1bdc>:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  x_forecast = np.array(df.drop(['Prediction'],1))[-forecast_out:]


In [40]:
#Prediction for the next " 'n' = forecast_out" days

lr_prediction = lr.predict(x_forecast)
print(lr_prediction)

svr_prediction = svr_rbf.predict(x_forecast)
print(svr_prediction)

[177.31839342 183.77249126 184.21830022 181.58397457 180.21615164
 182.14123576 183.23549411 187.59226345 189.25391501 185.73810347
 182.55664865 180.14522748 180.83420496 184.66410917 184.03592383
 188.01780836 186.62972139 189.55787566 189.08167064 186.16364839
 188.5041454  188.16978869 189.41602736 176.72060414 172.25238256
 173.50875326 168.94934348 163.37673153 164.05557699 156.08167589]
[177.91910713 181.42887501 181.28570489 175.36123969 172.62965072
 177.40384527 180.86129679 181.11339217 178.93583791 180.60047115
 178.94862835 172.61577633 173.32937869 180.86291141 181.3970526
 180.29001119 181.48982508 179.36984345 178.84516819 181.01739885
 179.31673628 179.96497542 179.1254103  177.17787495 172.58140744
 176.12660257 172.22786615 167.65548115 166.63742208 157.55715888]
